# Creating General ANCOR Dataset

In [ ]:
from ancor2dataset import chains2excel
import glob
import pandas as pd

PATH_ANCOR="../../DISTRIB_ANCOR/"
SUB_CORPRA=["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]

for SUB_CORPUS in SUB_CORPRA:

    PATH_ANCOR_SUB_CORPUS=PATH_ANCOR+SUB_CORPUS+'/'

    corpus_files=[file[file.rfind("/")+1:-4] for file in glob.glob(PATH_ANCOR_SUB_CORPUS+"annotation_integree/*.xml")]

    
    list_dataframes=[]


    for file in corpus_files:

        dataset = chains2excel(PATH_ANCOR)
        print("Starting",file)
        dataset.set_file_config(SUB_CORPUS, file)
        dataset.generate_json_mentions()
        dataset.remove_empty_json_mentions()
        dataset.generate_json_relations()
        if dataset.json_relations=={}:
            continue
        dataset.generate_json_chains(coreference_type=["ANAPHORE"]) #"DIRECTE", "INDIRECTE", 
#         dataset.generate_pos_neg_pairs(neg_pairs_percentage=0.95,
#                                  strategy="normal")

        dataset.generate_pos_neg_pairs(strategy = "pairs_from_left_with_window_size", window_size=20)
    
        dataset.extend_with_order_pos_neg_pairs()
        dataset.generate_training_test_dataset(save_files=False)


        list_dataframes.append(dataset.coreference_pairs_dataframe)


    train_test_dataframe=pd.concat(list_dataframes,ignore_index=True)
    train_test_dataframe.to_excel("./ds_window_all/"+SUB_CORPUS+"_ds_window_20_ANAPHORE.xlsx")


# Dataset Splitting

In [ ]:
from model_training import dataset_splitter

# ds1=dataset_splitter("./ds_90_10_new/corpus_ANCOR_N90_P10.xlsx",
#                     "./ds_90_10_new/corpus_ANCOR_N90_P10_Train.xlsx",
#                     "./ds_90_10_new/corpus_ANCOR_N90_P10_Test.xlsx")

ds2=dataset_splitter("./ds_window_15/corpus_ANCOR_Window_5.xlsx",
                    "./ds_window_15/corpus_ANCOR_Window_5_Train.xlsx",
                   "./ds_window_15/corpus_ANCOR_Window_5_Test.xlsx")

# dict_files=ds1.dataset_splitter_by_file(sub_corpus_num=4, lower_rate=0.20,upper_rate=0.22, files_num=84)
dict_files=ds2.dataset_splitter_by_dict(dict_files["sub_corpus_files_train"],dict_files["sub_corpus_files_test"])


# Model Training

In [ ]:
from model_training import model_trainer

model=model_trainer( "./ds_window_all/corpus_ANCOR_Window_ANAPHORE_Train.xlsx",
                     "./ds_window_5/corpus_ANCOR_Window_5_Test.xlsx",
                     "./ds_window_all/corpus_ANCOR_Window_Full_ANAPHORE_over_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                           'DISTANCE_MENTION','DISTANCE_WORD',
                           'DISTANCE_CHAR']

model.convert_columns_to_numeric()
model.train_model_random_forest(model_name="./Optimized_Models/Random_Forest_ANCOR_Window_ANAPHORE.model",max_depth=10, random_state=0,n_estimators=250)

# Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()